In [1]:
import os
from glob import glob
import json
import SimpleITK as sitk
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from pathlib import Path
import math


np.set_printoptions(precision=4, suppress=True)

In [2]:
path_outputs = "/home/ceballosarroyo.a/workspace/medical/cta-det2/outputs/adeform_decoder_only_non_rec_crop_vessel_pe_gpe_v2_EXT/inference_66k/predict.csv"
path_outputs_int = "/home/ceballosarroyo.a/workspace/medical/cta-det2/outputs/adeform_decoder_only_non_rec_crop_vessel_pe_gpe_v2/inference_56k/predict.csv"

In [17]:
scans_to_save = [
    "ExtA0015",
    "ExtA0045",
    "ExtA0022",
    "ExtB0017",
    "ExtB0043",
    "ExtA0020",
    "ExtB0047",
    "ExtB0042",
]

healthy_patients_external_fp = []


healthy_patients_internal_fp = [  # healthy
    "Ts0019.nii.gz",
    "Ts0021.nii.gz",
    "Ts0050.nii.gz",
    "Ts0060.nii.gz",
    "Ts0100.nii.gz",
    "Ts0110.nii.gz",
]

sick_patients_internal_fp = [  # sick
    "Ts0011.nii.gz",
    "Ts0052.nii.gz",
    "Ts0075.nii.gz",
    "Ts0084.nii.gz",
    "Ts0098.nii.gz",
]

patients_internal_fn = [
    "Ts0011.nii.gz",
    "Ts0061.nii.gz",
    "Ts0065.nii.gz",
    "Ts0111.nii.gz",
    "Ts0133.nii.gz",
    "Ts0136.nii.gz",
    "Ts0143.nii.gz",
]

all_scans_internal = (
    healthy_patients_internal_fp + sick_patients_internal_fp + patients_internal_fn
)

all_scans_external = healthy_patients_external_fp

threshold = 0.8

# combine
df = pd.concat([pd.read_csv(path_outputs), pd.read_csv(path_outputs_int)])
df = df[df["probability"] > threshold]
print(len(df))

path_metadata = "/home/ceballosarroyo.a/workspace/medical/cta-det2/labels/metadata/external_crop_meta.json"
path_metadata_internal = "/home/ceballosarroyo.a/workspace/medical/cta-det2/labels/metadata/internal_test_meta_crop.json"
path_labels = Path("/work/vig/Datasets/aneurysm/external/crop_0.4_label")
path_labels_int = Path("/work/vig/Datasets/aneurysm/internal_test/crop_0.4_label")
path_vessel_int = Path("/work/vig/Datasets/aneurysm/internal_test/crop_0.4_vessel_v2")
path_vessel = Path("/work/vig/Datasets/aneurysm/external/crop_0.4_vessel_v2")
scans_to_save = [str(path_labels_int / scan) for scan in all_scans_internal]
vessel_paths = [str(path_vessel_int / scan) for scan in all_scans_internal]
healthy_patients = healthy_patients_external_fp + healthy_patients_internal_fp

meta = json.load(open(path_metadata))
meta_internal = json.load(open(path_metadata_internal))

# merge meta and meta_internal
meta.update(meta_internal)

407


In [18]:
vessel_paths

['/work/vig/Datasets/aneurysm/internal_test_0.4_crop_vessel_v2/Ts0019.nii.gz',
 '/work/vig/Datasets/aneurysm/internal_test_0.4_crop_vessel_v2/Ts0021.nii.gz',
 '/work/vig/Datasets/aneurysm/internal_test_0.4_crop_vessel_v2/Ts0050.nii.gz',
 '/work/vig/Datasets/aneurysm/internal_test_0.4_crop_vessel_v2/Ts0060.nii.gz',
 '/work/vig/Datasets/aneurysm/internal_test_0.4_crop_vessel_v2/Ts0100.nii.gz',
 '/work/vig/Datasets/aneurysm/internal_test_0.4_crop_vessel_v2/Ts0110.nii.gz',
 '/work/vig/Datasets/aneurysm/internal_test_0.4_crop_vessel_v2/Ts0011.nii.gz',
 '/work/vig/Datasets/aneurysm/internal_test_0.4_crop_vessel_v2/Ts0052.nii.gz',
 '/work/vig/Datasets/aneurysm/internal_test_0.4_crop_vessel_v2/Ts0075.nii.gz',
 '/work/vig/Datasets/aneurysm/internal_test_0.4_crop_vessel_v2/Ts0084.nii.gz',
 '/work/vig/Datasets/aneurysm/internal_test_0.4_crop_vessel_v2/Ts0098.nii.gz',
 '/work/vig/Datasets/aneurysm/internal_test_0.4_crop_vessel_v2/Ts0011.nii.gz',
 '/work/vig/Datasets/aneurysm/internal_test_0.4_crop

In [19]:
folder_name = "outputs_review_v2"
# create folder name if it doesn't exist

if not os.path.exists(folder_name):
    os.makedirs(folder_name)
    os.makedirs(f"{folder_name}/healthy_for_fp_review")
    os.makedirs(f"{folder_name}/sick_for_both_fp_and_tp_review")

for j, scan in enumerate(scans_to_save):

    scan_name = Path(scan).name

    img_header = sitk.ReadImage(str(scan))
    vessel_header = sitk.ReadImage(vessel_paths[j])
    # vessel_header
    img = sitk.GetArrayFromImage(img_header)
    model_predictions = df[df["seriesuid"] == scan_name]
    pred_mask = np.zeros_like(img, dtype=np.uint8)
    for i, row in model_predictions.iterrows():
        metadata = meta[scan_name]
        seriesuid, prob, z, y, x, d, h, w = (
            row["seriesuid"],
            row["probability"],
            row["coordZ"],
            row["coordY"],
            row["coordX"],
            row["d"],
            row["h"],
            row["w"],
        )
        w = w * metadata["spacing"][0]
        h = h * metadata["spacing"][1]
        d = d * metadata["spacing"][2]
        sz, ez = int(z - d / 2), int(z + d / 2)
        sy, ey = int(y - h / 2), int(y + h / 2)
        sx, ex = int(x - w / 2), int(x + w / 2)
        if scan_name not in healthy_patients:
            print(scan_name, "not healthy")
            pred_mask[sz:ez, sy:ey, sx:ex] = int(np.floor(100 * prob))
        elif prob >= 0.8:
            pred_mask[sz:ez, sy:ey, sx:ex] = int(np.floor(100 * prob))

    new_header = sitk.GetImageFromArray(pred_mask)
    new_header.CopyInformation(img_header)

    if scan_name not in healthy_patients:
        sitk.WriteImage(
            new_header,
            f"{folder_name}/sick_for_both_fp_and_tp_review/{scan_name.replace('.nii.gz', '_pred.nii.gz')}",
        )
        sitk.WriteImage(
            img_header,
            f"{folder_name}/sick_for_both_fp_and_tp_review/{scan_name.replace('.nii.gz', '_label.nii.gz')}",
        )
        sitk.WriteImage(
            vessel_header,
            f"{folder_name}/sick_for_both_fp_and_tp_review/{scan_name.replace('.nii.gz', '_vessel.nii.gz')}",
        )

    else:
        sitk.WriteImage(
            new_header,
            f"{folder_name}/healthy_for_fp_review/{scan_name.replace('.nii.gz', '_pred.nii.gz')}",
        )
        sitk.WriteImage(
            vessel_header,
            f"{folder_name}/healthy_for_fp_review/{scan_name.replace('.nii.gz', '_vessel.nii.gz')}",
        )

Ts0011.nii.gz not healthy
Ts0011.nii.gz not healthy
Ts0011.nii.gz not healthy
Ts0011.nii.gz not healthy
Ts0011.nii.gz not healthy
Ts0052.nii.gz not healthy
Ts0052.nii.gz not healthy
Ts0052.nii.gz not healthy
Ts0052.nii.gz not healthy
Ts0052.nii.gz not healthy
Ts0075.nii.gz not healthy
Ts0075.nii.gz not healthy
Ts0075.nii.gz not healthy
Ts0075.nii.gz not healthy
Ts0084.nii.gz not healthy
Ts0084.nii.gz not healthy
Ts0084.nii.gz not healthy
Ts0084.nii.gz not healthy
Ts0084.nii.gz not healthy
Ts0098.nii.gz not healthy
Ts0098.nii.gz not healthy
Ts0098.nii.gz not healthy
Ts0098.nii.gz not healthy
Ts0011.nii.gz not healthy
Ts0011.nii.gz not healthy
Ts0011.nii.gz not healthy
Ts0011.nii.gz not healthy
Ts0011.nii.gz not healthy
Ts0065.nii.gz not healthy
Ts0065.nii.gz not healthy
Ts0111.nii.gz not healthy
Ts0111.nii.gz not healthy
Ts0133.nii.gz not healthy
Ts0133.nii.gz not healthy
Ts0133.nii.gz not healthy
Ts0133.nii.gz not healthy
Ts0136.nii.gz not healthy
Ts0136.nii.gz not healthy


In [12]:
metadata

{'origin': [-122.759765625, -255.759765625, -397.0],
 'spacing': [0.4000000059604645, 0.4000000059604645, 0.4000000059604645]}